In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.classifier import Classifier
from src.utils import cal_metric, print_results
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np

/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
data_dir = '../../Data/LISA/Federated_Data/Preprocessed_Data/Kia/1/'
ckpt_files = '../save/lightning_logs/version_0/checkpoints/epoch=49-step=32250.ckpt'
model = Classifier.load_from_checkpoint(ckpt_files, num_classes=3, data_dir=data_dir)
transform = None
test_dataset = CANDataset(root_dir=data_dir, is_train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, 
                        pin_memory=True, sampler=None)
trainer = pl.Trainer(enable_checkpointing=False, logger=False)
results = trainer.predict(model, dataloaders=test_loader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [10]:
labels = np.concatenate([x['labels'] for x in results])
preds = np.concatenate([x['preds'] for x in results])
cm, metrics = cal_metric(labels, preds)

In [11]:
classes = ['Normal', 'Fuzzy', 'Replay']
print_results(metrics, classes)

	fnr	rec	pre	f1
Normal	0.0331	0.9997	0.9989	0.9993
Fuzzy	0.0	1.0	1.0	1.0
Replay	7.0312	0.9297	0.9781	0.9533
